In [8]:
%matplotlib inline

import h5py
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import Callback as cbs
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import numpy as np
import pandas as pd
import os
from sklearn import model_selection, preprocessing, metrics
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
import time
import csv
import pickle
import warnings

#warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [9]:
RAW_DATA_PATH = '/kaggle/dev/mercedes-benz-greener-manufacturing-data/raw_data'
DATA_PATH = '/kaggle/dev/mercedes-benz-greener-manufacturing-data'
TRAIN_DATA = os.path.join(RAW_DATA_PATH, 'train.csv')
TEST_DATA = os.path.join(RAW_DATA_PATH, 'test.csv')
SAMPLE_SUBMISSION = os.path.join(RAW_DATA_PATH, 'sample_submission.csv')
SUBMISSION_PATH = os.path.join(DATA_PATH, 'submissions')
MODELS_PATH_NN = os.path.join(DATA_PATH, 'models/nn/')

In [10]:
train_df = pd.read_csv(TRAIN_DATA)
test_df = pd.read_csv(TEST_DATA)
sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION)

In [11]:
filter_outliers = False
filter_XO_X8 = True
dnn_use_augment_features = True

# Preprocess data
for column in train_df.columns:
    cardinality = len(np.unique(train_df[column]))
    if cardinality == 1:
        train_df.drop(column, axis=1, inplace=True)
        test_df.drop(column, axis=1, inplace=True)

x0_x8 = ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]
original_train_columns = sorted(list(set(train_df.columns) - set(['ID', 'y'])))

for f in ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]:
    lbl = preprocessing.LabelEncoder()
    train_values = set(train_df[f].values)
    test_values = set(test_df[f].values)
    all_values = list(train_values | test_values)
    lbl.fit(all_values) 
    train_df[f] = lbl.transform(list(train_df[f].values))
    test_df[f] = lbl.transform(list(test_df[f].values))

if filter_outliers:
    print('Filtering outliers')
    # Filter out outlier y = 265.32
    train_df = train_df[train_df.y < 200]
    
if filter_XO_X8:
    print('Filtering XO-X8')
    train_df = train_df[list(set(train_df.columns) - set(x0_x8))]
    test_df = test_df[list(set(test_df.columns) - set(x0_x8))]
    
print('train_df.shape', train_df.shape)
print('test_df.shape', test_df.shape)

Filtering XO-X8
train_df.shape (4209, 358)
test_df.shape (4209, 357)


In [12]:
n_comp = 10

# tSVD
# tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
# tsvd_results_train = tsvd.fit_transform(train_df.drop(["y"], axis=1))
# tsvd_results_test = tsvd.transform(test_df)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train_df.drop(["y"], axis=1))
pca2_results_test = pca.transform(test_df)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train_df.drop(["y"], axis=1))
ica2_results_test = ica.transform(test_df)

# GRP
# grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
# grp_results_train = grp.fit_transform(train_df.drop(["y"], axis=1))
# grp_results_test = grp.transform(test_df)

# # SRP
# srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
# srp_results_train = srp.fit_transform(train_df.drop(["y"], axis=1))
# srp_results_test = srp.transform(test_df)

# Append decomposition components to datasets
for i in range(0, n_comp):
    train_df['pca_' + str(i)] = pca2_results_train[:, i]
    test_df['pca_' + str(i)] = pca2_results_test[:, i]

    train_df['ica_' + str(i)] = ica2_results_train[:, i]
    test_df['ica_' + str(i)] = ica2_results_test[:, i]

#     train_df['tsvd_' + str(i)] = tsvd_results_train[:, i]
#     test_df['tsvd_' + str(i)] = tsvd_results_test[:, i]

#     train_df['grp_' + str(i)] = grp_results_train[:, i]
#     test_df['grp_' + str(i)] = grp_results_test[:, i]

#     train_df['srp_' + str(i)] = srp_results_train[:, i]
#     test_df['srp_' + str(i)] = srp_results_test[:, i]

augmented_train_columns = sorted(list(set(train_df.columns) - set(['ID', 'y'])))
print('original columns', len(original_train_columns))
print('augmented columns', len(augmented_train_columns))
print('train_df.shape', train_df.shape)
print('test_df.shape', test_df.shape)

original columns 364
augmented columns 376
train_df.shape (4209, 378)
test_df.shape (4209, 377)


In [13]:
train_df.to_csv(os.path.join(DATA_PATH, 'train_df_encoded.csv'))
test_df.to_csv(os.path.join(DATA_PATH, 'test_df_encoded.csv'))

In [14]:
X = train_df.drop(["ID", "y"], axis=1)
Y = train_df['y'].values
train_X, val_X, train_Y, val_Y = model_selection.train_test_split(X, Y, test_size=0.2, random_state=29)
print('X.shape', X.shape)
print('Y.shape', Y.shape)
print('train_X.shape', train_X.shape)
print('train_Y.shape', train_Y.shape)
print('val_X.shape', val_X.shape)
print('val_Y.shape', val_Y.shape)

X.shape (4209, 376)
Y.shape (4209,)
train_X.shape (3367, 376)
train_Y.shape (3367,)
val_X.shape (842, 376)
val_Y.shape (842,)


In [ ]:
BATCH_SIZE = 32
timestamp = str(int(time.time()))

os.mkdir(MODELS_PATH_NN + timestamp)

print('Saving models to', MODELS_PATH_NN + timestamp)

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res / (SS_tot + K.epsilon()))

class LossHistory(cbs):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

callbacks = [
    LossHistory(),
#     EarlyStopping(
#         monitor='val_r2_keras',
#         min_delta=0.001
#         patience=20,
#         mode='max',
#         verbose=0),
    ModelCheckpoint(
        MODELS_PATH_NN + timestamp + '/' + '{epoch:04d}-{r2_keras:.6f}-{val_r2_keras:.6f}.hdf5', 
        monitor='r2_keras', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

input_dims = train_X.shape[1]

# This returns a tensor
inputs = Input(shape=(input_dims,))

# a layer instance is callable on a tensor, and returns a tensor

print('inputs.shape', inputs.shape)

x = Dense(input_dims, activation='relu')(inputs)
x = BatchNormalization()(x)
#x = Activation('relu')(x)
x = Dropout(0.2)(x)

x = Dense(input_dims, activation='relu')(inputs)
x = BatchNormalization()(x)
#x = Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(input_dims, activation='relu')(inputs)
x = BatchNormalization()(x)
#x = Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(input_dims, activation='relu')(inputs)
x = BatchNormalization()(x)
#x = Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(input_dims//2, activation='relu')(inputs)
x = BatchNormalization()(x)
#x = Activation('relu')(x)
x = Dropout(0.3)(x)

# Output Layer.
outputs = Dense(1)(x)

print('outputs.shape', outputs.shape)

adamOptimizer = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=adamOptimizer,
              loss='mean_squared_error', #'mean_squared_logarithmic_error',
              metrics=[r2_keras, 'mae', 'mean_squared_logarithmic_error'])

k_X, k_Y = train_X.copy().values, train_Y
k_val_X = val_X.copy().values

model.fit([k_X], [k_Y], epochs=5000, batch_size=BATCH_SIZE, validation_data=(k_val_X, val_Y), callbacks=callbacks)

Saving models to /kaggle/dev/mercedes-benz-greener-manufacturing-data/models/nn/1498249429
inputs.shape (?, 376)
outputs.shape (?, 1)
Train on 3367 samples, validate on 842 samples
Epoch 1/5000
3367/3367 [==============================] - 18s - loss: 10150.3372 - r2_keras: -74.4371 - mean_absolute_error: 99.9737 - mean_squared_logarithmic_error: 17.1502 - val_loss: 10265.6888 - val_r2_keras: -70.7231 - val_mean_absolute_error: 100.3896 - val_mean_squared_logarithmic_error: 15.7443
Epoch 2/5000
3367/3367 [==============================] - 0s - loss: 9896.0223 - r2_keras: -70.9553 - mean_absolute_error: 98.7127 - mean_squared_logarithmic_error: 13.8316 - val_loss: 9977.3392 - val_r2_keras: -68.6913 - val_mean_absolute_error: 98.9504 - val_mean_squared_logarithmic_error: 11.3023
Epoch 3/5000
3367/3367 [==============================] - 0s - loss: 9379.6369 - r2_keras: -68.7314 - mean_absolute_error: 96.0846 - mean_squared_logarithmic_error: 9.5612 - val_loss: 9073.7223 - val_r2_keras: -62

3367/3367 [==============================] - 0s - loss: 142.1181 - r2_keras: 0.0051 - mean_absolute_error: 8.9156 - mean_squared_logarithmic_error: 0.0133 - val_loss: 127.6468 - val_r2_keras: 0.3859 - val_mean_absolute_error: 7.5839 - val_mean_squared_logarithmic_error: 0.0093
Epoch 29/5000
3367/3367 [==============================] - 0s - loss: 139.2425 - r2_keras: 0.0271 - mean_absolute_error: 8.8363 - mean_squared_logarithmic_error: 0.0128 - val_loss: 126.2485 - val_r2_keras: 0.4090 - val_mean_absolute_error: 6.9998 - val_mean_squared_logarithmic_error: 0.0090
Epoch 30/5000
3367/3367 [==============================] - 0s - loss: 135.5082 - r2_keras: 0.0474 - mean_absolute_error: 8.7356 - mean_squared_logarithmic_error: 0.0125 - val_loss: 116.9723 - val_r2_keras: 0.4632 - val_mean_absolute_error: 6.6342 - val_mean_squared_logarithmic_error: 0.0082
Epoch 31/5000
3367/3367 [==============================] - 0s - loss: 127.7977 - r2_keras: 0.1220 - mean_absolute_error: 8.5775 - mean_squ

3367/3367 [==============================] - 0s - loss: 118.4555 - r2_keras: 0.1781 - mean_absolute_error: 8.1582 - mean_squared_logarithmic_error: 0.0108 - val_loss: 117.5958 - val_r2_keras: 0.4703 - val_mean_absolute_error: 6.2047 - val_mean_squared_logarithmic_error: 0.0081
Epoch 57/5000
3367/3367 [==============================] - 0s - loss: 124.7523 - r2_keras: 0.1197 - mean_absolute_error: 8.2842 - mean_squared_logarithmic_error: 0.0114 - val_loss: 107.7849 - val_r2_keras: 0.5352 - val_mean_absolute_error: 5.8013 - val_mean_squared_logarithmic_error: 0.0073
Epoch 58/5000
3367/3367 [==============================] - 0s - loss: 121.7237 - r2_keras: 0.1685 - mean_absolute_error: 8.2441 - mean_squared_logarithmic_error: 0.0111 - val_loss: 123.1783 - val_r2_keras: 0.4319 - val_mean_absolute_error: 6.6699 - val_mean_squared_logarithmic_error: 0.0087
Epoch 59/5000
3367/3367 [==============================] - 0s - loss: 124.9788 - r2_keras: 0.1339 - mean_absolute_error: 8.3288 - mean_squ

3367/3367 [==============================] - 0s - loss: 111.7393 - r2_keras: 0.2352 - mean_absolute_error: 7.8206 - mean_squared_logarithmic_error: 0.0102 - val_loss: 107.8970 - val_r2_keras: 0.5285 - val_mean_absolute_error: 5.8998 - val_mean_squared_logarithmic_error: 0.0073
Epoch 85/5000
3367/3367 [==============================] - 0s - loss: 111.1217 - r2_keras: 0.2330 - mean_absolute_error: 7.7890 - mean_squared_logarithmic_error: 0.0100 - val_loss: 107.3967 - val_r2_keras: 0.5316 - val_mean_absolute_error: 5.9834 - val_mean_squared_logarithmic_error: 0.0073
Epoch 86/5000
3367/3367 [==============================] - 0s - loss: 104.0292 - r2_keras: 0.2734 - mean_absolute_error: 7.6065 - mean_squared_logarithmic_error: 0.0095 - val_loss: 110.4670 - val_r2_keras: 0.5119 - val_mean_absolute_error: 6.1633 - val_mean_squared_logarithmic_error: 0.0076
Epoch 87/5000
3367/3367 [==============================] - 0s - loss: 109.5160 - r2_keras: 0.2514 - mean_absolute_error: 7.7329 - mean_squ

3367/3367 [==============================] - 0s - loss: 105.8610 - r2_keras: 0.2833 - mean_absolute_error: 7.6051 - mean_squared_logarithmic_error: 0.0095 - val_loss: 105.6758 - val_r2_keras: 0.5464 - val_mean_absolute_error: 5.7495 - val_mean_squared_logarithmic_error: 0.0071
Epoch 113/5000
3367/3367 [==============================] - 0s - loss: 102.7671 - r2_keras: 0.2923 - mean_absolute_error: 7.4347 - mean_squared_logarithmic_error: 0.0092 - val_loss: 109.2584 - val_r2_keras: 0.5254 - val_mean_absolute_error: 5.8506 - val_mean_squared_logarithmic_error: 0.0074
Epoch 114/5000
3367/3367 [==============================] - 0s - loss: 105.7813 - r2_keras: 0.2598 - mean_absolute_error: 7.5138 - mean_squared_logarithmic_error: 0.0094 - val_loss: 105.9911 - val_r2_keras: 0.5439 - val_mean_absolute_error: 5.7558 - val_mean_squared_logarithmic_error: 0.0071
Epoch 115/5000
3367/3367 [==============================] - 0s - loss: 107.8942 - r2_keras: 0.2610 - mean_absolute_error: 7.5940 - mean_

...

3367/3367 [==============================] - 0s - loss: 76.3607 - r2_keras: 0.4655 - mean_absolute_error: 6.3428 - mean_squared_logarithmic_error: 0.0067 - val_loss: 102.9981 - val_r2_keras: 0.5584 - val_mean_absolute_error: 5.6848 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2854/5000
3367/3367 [==============================] - 0s - loss: 74.6836 - r2_keras: 0.4808 - mean_absolute_error: 6.2650 - mean_squared_logarithmic_error: 0.0066 - val_loss: 104.7738 - val_r2_keras: 0.5480 - val_mean_absolute_error: 5.8487 - val_mean_squared_logarithmic_error: 0.0071
Epoch 2855/5000
3367/3367 [==============================] - 0s - loss: 73.0542 - r2_keras: 0.5000 - mean_absolute_error: 6.2118 - mean_squared_logarithmic_error: 0.0064 - val_loss: 109.5344 - val_r2_keras: 0.5192 - val_mean_absolute_error: 5.9697 - val_mean_squared_logarithmic_error: 0.0075
Epoch 2856/5000
3367/3367 [==============================] - 0s - loss: 73.0215 - r2_keras: 0.5040 - mean_absolute_error: 6.1567 - mean_s

3367/3367 [==============================] - 0s - loss: 73.5726 - r2_keras: 0.4943 - mean_absolute_error: 6.2250 - mean_squared_logarithmic_error: 0.0064 - val_loss: 103.8343 - val_r2_keras: 0.5491 - val_mean_absolute_error: 5.7134 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2882/5000
3367/3367 [==============================] - 0s - loss: 75.4520 - r2_keras: 0.4810 - mean_absolute_error: 6.1978 - mean_squared_logarithmic_error: 0.0066 - val_loss: 103.1651 - val_r2_keras: 0.5503 - val_mean_absolute_error: 5.8909 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2883/5000
3367/3367 [==============================] - 0s - loss: 76.0147 - r2_keras: 0.4876 - mean_absolute_error: 6.2465 - mean_squared_logarithmic_error: 0.0066 - val_loss: 104.0123 - val_r2_keras: 0.5525 - val_mean_absolute_error: 5.6516 - val_mean_squared_logarithmic_error: 0.0070
Epoch 2884/5000
3367/3367 [==============================] - 0s - loss: 73.2633 - r2_keras: 0.4987 - mean_absolute_error: 6.2230 - mean_s

3367/3367 [==============================] - 0s - loss: 73.7518 - r2_keras: 0.4980 - mean_absolute_error: 6.2082 - mean_squared_logarithmic_error: 0.0064 - val_loss: 103.6350 - val_r2_keras: 0.5519 - val_mean_absolute_error: 5.7091 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2910/5000
3367/3367 [==============================] - 0s - loss: 75.2909 - r2_keras: 0.4899 - mean_absolute_error: 6.2522 - mean_squared_logarithmic_error: 0.0066 - val_loss: 105.0737 - val_r2_keras: 0.5488 - val_mean_absolute_error: 5.6042 - val_mean_squared_logarithmic_error: 0.0070
Epoch 2911/5000
3367/3367 [==============================] - 0s - loss: 77.0610 - r2_keras: 0.4696 - mean_absolute_error: 6.3274 - mean_squared_logarithmic_error: 0.0068 - val_loss: 104.0074 - val_r2_keras: 0.5513 - val_mean_absolute_error: 5.7682 - val_mean_squared_logarithmic_error: 0.0070
Epoch 2912/5000
3367/3367 [==============================] - 0s - loss: 73.8083 - r2_keras: 0.5039 - mean_absolute_error: 6.1999 - mean_s

3367/3367 [==============================] - 0s - loss: 71.9545 - r2_keras: 0.5144 - mean_absolute_error: 6.1689 - mean_squared_logarithmic_error: 0.0063 - val_loss: 103.4339 - val_r2_keras: 0.5538 - val_mean_absolute_error: 5.6953 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2937/5000
3367/3367 [==============================] - 0s - loss: 76.0975 - r2_keras: 0.4780 - mean_absolute_error: 6.4235 - mean_squared_logarithmic_error: 0.0067 - val_loss: 104.5879 - val_r2_keras: 0.5436 - val_mean_absolute_error: 5.7132 - val_mean_squared_logarithmic_error: 0.0070
Epoch 2938/5000
3367/3367 [==============================] - 0s - loss: 75.9728 - r2_keras: 0.4747 - mean_absolute_error: 6.2855 - mean_squared_logarithmic_error: 0.0067 - val_loss: 103.3748 - val_r2_keras: 0.5497 - val_mean_absolute_error: 5.7984 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2939/5000
3367/3367 [==============================] - 0s - loss: 73.4579 - r2_keras: 0.4914 - mean_absolute_error: 6.1565 - mean_s

3367/3367 [==============================] - 0s - loss: 76.1742 - r2_keras: 0.4692 - mean_absolute_error: 6.3216 - mean_squared_logarithmic_error: 0.0067 - val_loss: 106.2833 - val_r2_keras: 0.5377 - val_mean_absolute_error: 5.8461 - val_mean_squared_logarithmic_error: 0.0072
Epoch 2965/5000
3367/3367 [==============================] - 0s - loss: 73.6447 - r2_keras: 0.4990 - mean_absolute_error: 6.2463 - mean_squared_logarithmic_error: 0.0064 - val_loss: 102.7668 - val_r2_keras: 0.5549 - val_mean_absolute_error: 5.7439 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2966/5000
3367/3367 [==============================] - 0s - loss: 74.1181 - r2_keras: 0.4958 - mean_absolute_error: 6.2409 - mean_squared_logarithmic_error: 0.0065 - val_loss: 102.8185 - val_r2_keras: 0.5586 - val_mean_absolute_error: 5.6643 - val_mean_squared_logarithmic_error: 0.0069
Epoch 2967/5000
3367/3367 [==============================] - 0s - loss: 76.6624 - r2_keras: 0.4628 - mean_absolute_error: 6.3106 - mean_s

In [52]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res / (SS_tot + K.epsilon()))

model_id1 = '1498183323/3405-0.556942-0.540529.hdf5'
model_id2 = '1498183323/1174-0.479140-0.568797.hdf5'
model_id3 = '1498183323/0465-0.443299-0.577629.hdf5'

k_test_X = test_df.drop(["ID"], axis=1).copy().values
model = load_model(MODELS_PATH_NN + model_id3, custom_objects={'r2_keras': r2_keras})
test_Y = model.predict(k_test_X).ravel()
test_Y

array([  83.27249146,   98.08628845,   59.55910492, ...,   92.20954895,
        107.00197601,   91.37447357], dtype=float32)

In [53]:
submission_df = test_df[['ID']]
submission_df['y'] = test_Y.tolist()
submission_df.to_csv(os.path.join(SUBMISSION_PATH, 'submission-' + str(int(time.time())) + '.csv'), index=False)
print('Generated submission ', os.path.join(SUBMISSION_PATH, 'submission-' + str(int(time.time())) + '.csv'))

Generated submission  /kaggle/dev/mercedes-benz-greener-manufacturing-data/submissions/submission-1498187853.csv


/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
